In [3]:
### Import math and visualization libraries

# import required libraries
import numpy as np
import plotly.graph_objects as go


ModuleNotFoundError: No module named 'numpy'

In [4]:
### Define contants and variables

# define loop variables
steps = 500
float_rounding = 4

# define constants
sessions = 5
cell_number = 1000000
lethal_single_dose = 30
dose_per_session = 12

# define variables
coupling = 0
coupling_increment = 0.005
coupling_min = 0.3
coupling_max = 0.5
gamma = 3
gamma_increment = 0.01
gamma_min = 0
gamma_max = 100
tcp_min = 0.79
tcp_max = 0.81

# define lists to store results
result_coupling = []
result_gamma = []
result_tumor_control_probability = []

In [5]:
### Main loop

# store initial coupling to reset between each y loop
coupling_start = coupling

y_steps_count = 0
# GAMMA LOOP
while y_steps_count<steps:

    coupling = coupling_start
    x_steps_count = 0
    # COUPLING LOOP
    while x_steps_count<steps:
        # get normalized dose
        normalized_dose = dose_per_session/lethal_single_dose

        # preinitialize list of accumulated doses
        acc_doses = [None]*(sessions+1)

        # set first session dose (skip zero to match session number with list index)
        session_count = 1
        acc_doses[session_count] = normalized_dose

        # SESSION LOOP: iterate each session by acumulating the dose from previous session
        while session_count<sessions:
            session_count += 1
            prev_dose = acc_doses[session_count-1]
            try:
                acc_doses[session_count] = prev_dose+normalized_dose*((1-prev_dose)/(1-coupling*prev_dose))
            except ZeroDivisionError:
                acc_doses[session_count] = None
            except TypeError:
                acc_doses[session_count] = None

        # calculated cell survival from accumulated dose result
        try:
            cell_survival = np.power((1-acc_doses[session_count]), gamma)
        except TypeError:
            cell_survival = None

        # calculate tumor control probability with
        try:
            tumor_control_probability = np.exp(-(cell_number*cell_survival))
        except TypeError:
            tumor_control_probability = 0
        
        # change to 0 or 1 all result not between 0.0 and 1.0
        if tumor_control_probability < 0.0:
            tumor_control_probability = 0
        elif tumor_control_probability > 1.0:
            tumor_control_probability = 1

        # store values in result lists
        if coupling_min < coupling < coupling_max and gamma_min < gamma < gamma_max and tcp_min < tumor_control_probability < tcp_max:
            result_coupling.append(coupling)
            result_gamma.append(gamma)
            result_tumor_control_probability.append(tumor_control_probability)
        

        # increment coupling and step
        coupling = round(coupling+coupling_increment,float_rounding)
        x_steps_count += 1

    # increment gamma and step
    gamma = round(gamma+gamma_increment,float_rounding)
    y_steps_count += 1

NameError: name 'np' is not defined

In [49]:
### Display result

fig = go.Figure(data=[go.Scatter3d(
    x=result_coupling,
    y=result_gamma,
    z=result_tumor_control_probability,
    mode='markers',
        marker=dict(
        size=1,
        opacity=1
    )
    )
])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()